# Import Library & Setup

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, callbacks
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

# ML
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score
)
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.ensemble import RandomForestClassifier

# Viz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

np.random.seed(42)
tf.random.set_seed(42)

print('Libraries loaded')
print(f'Time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

Libraries loaded
Time: 2026-02-06 11:20:24


# Load Data

In [13]:
file = '../data/road_dataset.csv'

try:
    df = pd.read_csv(file)
    print(f'Loaded: {df.shape[0]} shoes × {df.shape[1]} columns')
    display(df.head())
except FileNotFoundError:
    print(f"WARNING: '{file}' not found.")
    print("Please upload the correct dataset file to run with actual data.")

Loaded: 443 shoes × 45 columns


,brand,name,lightweight,rocker,orthotic_friendly,removable_insole,pace_daily_running,pace_tempo,pace_competition,arch_neutral,...,heel_stiff_flexible,heel_stiff_moderate,heel_stiff_stiff,plate_rock,plate_carbon,heel_lab_mm,forefoot_lab_mm,season_summer,season_winter,season_all
0,brooks,launch 9,1,0,1,1,1,1,0,1,...,1,0,0,0,0,32.4,23.0,0,0,0
1,brooks,levitate 6,0,0,1,1,1,0,0,1,...,0,1,0,0,0,34.3,26.6,1,0,1
2,adidas,4dfwd,0,0,1,1,1,0,0,1,...,1,0,0,0,0,33.3,24.4,0,0,1
3,adidas,4dfwd 2,0,0,1,1,1,0,0,1,...,0,1,0,0,0,31.8,21.2,0,0,1
4,adidas,4dfwd 3,0,0,1,1,1,0,0,1,...,1,0,0,0,0,32.6,22.7,0,0,1


In [14]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 45 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                443 non-null    str    
 1   name                 443 non-null    str    
 2   lightweight          443 non-null    int64  
 3   rocker               443 non-null    int64  
 4   orthotic_friendly    443 non-null    int64  
 5   removable_insole     443 non-null    int64  
 6   pace_daily_running   443 non-null    int64  
 7   pace_tempo           443 non-null    int64  
 8   pace_competition     443 non-null    int64  
 9   arch_neutral         443 non-null    int64  
 10  arch_stability       443 non-null    int64  
 11  weight_lab_oz        443 non-null    float64
 12  drop_lab_mm          443 non-null    float64
 13  strike_heel          443 non-null    int64  
 14  strike_mid           443 non-null    int64  
 15  strike_forefoot      443 non-null    int64  
 16  s

# Preprocessing

In [15]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

binary_cols = [col for col in numeric_cols if set(df[col].unique()).issubset({0, 1})]
continuous_cols = [col for col in numeric_cols if col not in binary_cols]

print(f'Features: {len(numeric_cols)} total')
print(f'  Binary     : {len(binary_cols)}')
print(f'  Continuous : {len(continuous_cols)}')

Features: 43 total
  Binary     : 35
  Continuous : 8


In [16]:
feature_cols = numeric_cols.copy()
X = df[feature_cols]

# Separate for proper scaling
X_binary = X[binary_cols].values
X_continuous = X[continuous_cols].values

# Scale continuous to 0-1 for neural network
scaler_continuous = MinMaxScaler()
X_continuous_scaled = scaler_continuous.fit_transform(X_continuous)

# Combine
X_combined = np.concatenate([X_binary, X_continuous_scaled], axis=1)

# Also standard scaling for traditional comparison
scaler_standard = StandardScaler()
X_standard = scaler_standard.fit_transform(X)

print(f'Neural input shape: {X_combined.shape}')
print(f'Range: [{X_combined.min():.6f}, {X_combined.max():.6f}]')

Neural input shape: (443, 43)
Range: [0.000000, 1.000000]


# Auto-Encoder

## Modelling

In [17]:
# Architecture
input_dim = X_combined.shape[1]
encoding_dims = [32, 16, 8]

# Encoder
input_layer = Input(shape=(input_dim,))
x = input_layer
for dim in encoding_dims:
    x = Dense(dim, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

latent = x

# Decoder
for dim in reversed(encoding_dims[:-1]):
    x = Dense(dim, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

output_layer = Dense(input_dim, activation='sigmoid')(x)

autoencoder = Model(input_layer, output_layer)
encoder = Model(input_layer, latent)

autoencoder.compile(
    optimizer=optimizers.Adam(0.001),
    loss='mse',
    metrics=['mae']
)

print('Autoencoder architecture:')
autoencoder.summary()

Autoencoder architecture:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 43)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 43)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,595 (17.95 KB)

 Trainable params: 4,387 (17.14 KB)

 Non-trainable params: 208 (832.00 B)

## Training

In [18]:
history = autoencoder.fit(
    X_combined, X_combined,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-5)
    ],
    verbose=0
)

print(f'Training done!')
print(f'Final loss: {history.history["loss"][-1]:.6f}')
print(f'  Val loss: {history.history["val_loss"][-1]:.6f}')

# Get latent representations
X_latent = encoder.predict(X_combined, verbose=0)
print(f'Latent space: {X_latent.shape} (8D embeddings)')

Training done!
Final loss: 0.107215
  Val loss: 0.093787
Latent space: (443, 8) (8D embeddings)


# Metrics Function

In [19]:
def calculate_cluster_purity(df, cluster_col, binary_cols):
    purity_by_cluster = {}
    for cid in df[cluster_col].unique():
        cdata = df[df[cluster_col] == cid]
        n = len(cdata)
        dominances = []
        for col in binary_cols:
            if col in cdata.columns:
                vc = cdata[col].value_counts()
                if len(vc) > 0:
                    dominances.append(vc.max() / n)
        purity_by_cluster[cid] = {'purity': np.mean(dominances) if dominances else 0, 'n': n}
    all_p = [v['purity'] for v in purity_by_cluster.values()]
    return {
        'by_cluster': purity_by_cluster,
        'mean_purity': np.mean(all_p),
        'min_purity': np.min(all_p),
        'max_purity': np.max(all_p)
    }

def calculate_cluster_stability(X, labels, model_func, n_iter=20):
    n = len(X)
    ari_scores = []
    for _ in range(n_iter):
        idx = np.random.choice(n, n, replace=True)
        boot_labels = model_func().fit_predict(X[idx])
        ari = adjusted_rand_score(labels[idx], boot_labels)
        ari_scores.append(ari)
    m = np.mean(ari_scores)
    return {
        'mean_ari': m,
        'std_ari': np.std(ari_scores),
        'stability_level': 'Excellent' if m > 0.8 else 'Good' if m > 0.6 else 'Moderate'
    }

def calculate_interpretability_score(df, cluster_col, binary_cols, threshold=0.75):
    scores = []
    for cid in df[cluster_col].unique():
        cdata = df[df[cluster_col] == cid]
        n = len(cdata)
        strong = sum(1 for col in binary_cols if col in cdata.columns and
                    (cdata[col].sum()/n > threshold or cdata[col].sum()/n < 1-threshold))
        scores.append(strong / len(binary_cols))
    return {'mean_interpretability': np.mean(scores), 'scores': scores}

def evaluate_clustering_comprehensive(X, labels, df_temp, model_func, binary_cols):
    sil = silhouette_score(X, labels)
    db = davies_bouldin_score(X, labels)
    ch = calinski_harabasz_score(X, labels)
    df_temp['cluster'] = labels
    purity = calculate_cluster_purity(df_temp, 'cluster', binary_cols)
    stability = calculate_cluster_stability(X, labels, model_func, 10)
    interp = calculate_interpretability_score(df_temp, 'cluster', binary_cols)

    sil_norm = (sil + 1) / 2
    db_norm = 1 / (1 + db)
    ch_norm = min(ch / 1000, 1)
    composite = (0.25*sil_norm + 0.20*db_norm + 0.15*ch_norm +
                 0.25*purity['mean_purity'] + 0.10*stability['mean_ari'] +
                 0.05*interp['mean_interpretability'])

    return {
        'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch,
        'purity': purity['mean_purity'], 'stability': stability['mean_ari'],
        'interpretability': interp['mean_interpretability'], 'composite_score': composite
    }

print('Metrics functions ready')

Metrics functions ready


# Model Selection

In [20]:
results = []

# Header Tabel
print(f"| {'K':^3} | {'Score':^8} | {'Sil.':^8} | {'DB':^8} | {'CH':^10} | {'Purity':^8} | {'Stab.':^8} | {'Interp':^8} |")
print(f"|{'-'*5}+{'-'*10}+{'-'*10}+{'-'*10}+{'-'*12}+{'-'*10}+{'-'*10}+{'-'*10}|")

for i in range(3, 10):
    np.random.seed(42)
    
    model_factory = lambda: KMeans(n_clusters=i, random_state=42, n_init=10)
    model = model_factory()
    labels = model.fit_predict(X_latent)

    metrics = evaluate_clustering_comprehensive(
        X_latent, labels, df.copy(),
        model_factory,
        binary_cols
    )

    # Simpan hasil
    results.append({
        'k': i,
        'model': model,
        'labels': labels,
        **metrics
    })

    # Print Baris Tabel
    print(f"| {i:^3} | {metrics['composite_score']:<8.6f} | {metrics['silhouette']:<6.6f} | "
          f"{metrics['davies_bouldin']:<6.6f} | {metrics['calinski_harabasz']:<8.6f} | "
          f"{metrics['purity']:<6.6f} | {metrics['stability']:<6.6f} | {metrics['interpretability']:<6.6f} |")

|  K  |  Score   |   Sil.   |    DB    |     CH     |  Purity  |  Stab.   |  Interp  |
|-----+----------+----------+----------+------------+----------+----------+----------|
|  3  | 0.593536 | 0.318839 | 1.222029 | 187.790422 | 0.818563 | 0.725302 | 0.666667 |
|  4  | 0.587487 | 0.308971 | 1.125864 | 189.624032 | 0.820910 | 0.621866 | 0.678571 |
|  5  | 0.631420 | 0.358737 | 0.977719 | 219.117526 | 0.824006 | 0.878682 | 0.674286 |
|  6  | 0.624365 | 0.364098 | 0.982893 | 212.041853 | 0.832453 | 0.768797 | 0.723810 |
|  7  | 0.647879 | 0.382525 | 0.910976 | 224.833769 | 0.844293 | 0.888721 | 0.734694 |
|  8  | 0.624270 | 0.348014 | 0.978091 | 221.571360 | 0.843982 | 0.741795 | 0.725000 |
|  9  | 0.625242 | 0.355770 | 0.988909 | 218.458118 | 0.846443 | 0.746432 | 0.723810 |


In [21]:
df_results = pd.DataFrame(results)
best_config = df_results.loc[df_results['composite_score'].idxmax()]

best_model = best_config['model']
best_labels = best_config['labels']
best_k = best_config['k']
X_for_clustering = X_latent

print(f'SELECTED BEST K: {best_k}')
print(f'   Silhouette      : {best_config["silhouette"]:.6f}')
print(f'   Composite Score : {best_config["composite_score"]:.6f}')

SELECTED BEST K: 7
   Silhouette      : 0.382525
   Composite Score : 0.647879


# Generate Cluster Label

In [ ]:
# Masukkan Cluster ke DataFrame
df['cluster'] = best_labels 

# Setup Grouping
bin_groups = {}
for col in binary_cols:
    parts = col.split('_')
    
    if len(parts) > 1:
        prefix = '_'.join(parts[:-1])
    else:
        prefix = col
        
    bin_groups.setdefault(prefix, []).append(col)

# Build Summary Data
rows = []
for cid in sorted(df['cluster'].unique()):
    subset = df[df['cluster'] == cid]
    n = len(subset)
    
    row = {'count': n, 'percentage': f"{n/len(df)*100:.1f}%"}

    # A. Continuous Columns: Langsung ambil mean
    for col in continuous_cols:
        row[col.lower()] = round(subset[col].mean(), 2)

    # B. Binary Groups
    for prefix, cols in bin_groups.items():
        # Hitung mean grup ini
        means = subset[cols].mean()
        best_col = means.idxmax()
        best_val = means.max()
        
        # Case 1: Multiple Variants
        if len(cols) > 1:
            header = prefix.lower()
            val_str = best_col.replace(f"{prefix}_", "").lower()
            row[header] = f"{val_str} ({best_val*100:.0f}%)"
            
        # Case 2: Standalone
        else:
            header = cols[0].lower()
            val_str = "yes" if best_val > 0.5 else "no"
            row[header] = f"{val_str} ({best_val*100:.0f}%)"

    rows.append(row)

# Create DataFrame & Fix Display
df_summary = pd.DataFrame(rows, index=sorted(df['cluster'].unique()))
df_summary.index.name = None 

print("Cluster Summary:")
display(df_summary)

Cluster Summary:


,count,percentage,weight_lab_oz,drop_lab_mm,toebox_durability,heel_durability,outsole_durability,breathability,heel_lab_mm,forefoot_lab_mm,...,arch,strike,softness,width,toebox,stiffness,torsional,heel_stiff,plate,season
0,12,2.7%,7.08,7.21,0.17,0.67,0.25,3.58,33.38,26.17,...,neutral (100%),mid (92%),soft (25%),narrow (100%),narrow (8%),stiff (75%),stiff (83%),flexible (92%),carbon (92%),all (75%)
1,109,24.6%,10.00,8.32,2.90,3.21,3.56,3.32,36.71,28.40,...,neutral (85%),mid (79%),balanced (58%),medium (64%),medium (60%),stiff (92%),stiff (74%),moderate (50%),carbon (9%),all (97%)
2,45,10.2%,7.99,5.45,2.44,2.49,2.53,3.58,27.61,22.20,...,neutral (96%),mid (96%),soft (56%),medium (51%),wide (42%),moderate (53%),flexible (76%),flexible (69%),carbon (2%),all (98%)
3,100,22.6%,9.70,8.87,0.12,0.16,0.02,1.39,32.95,24.08,...,neutral (85%),mid (66%),balanced (26%),narrow (51%),narrow (6%),stiff (77%),moderate (41%),moderate (26%),carbon (2%),all (43%)
4,39,8.8%,7.57,8.94,2.59,3.44,3.10,4.26,38.80,29.86,...,neutral (100%),heel (77%),soft (77%),narrow (54%),medium (59%),stiff (92%),stiff (97%),flexible (90%),carbon (97%),all (100%)
5,59,13.3%,9.91,9.92,3.25,3.54,3.73,3.24,37.81,27.89,...,neutral (58%),heel (90%),soft (59%),medium (86%),medium (88%),moderate (97%),stiff (97%),stiff (64%),rock (0%),all (95%)
6,79,17.8%,9.65,9.56,2.87,3.37,3.46,3.37,34.22,24.67,...,neutral (91%),heel (80%),balanced (58%),medium (76%),medium (72%),moderate (86%),moderate (66%),moderate (44%),rock (0%),all (96%)
